In [1]:
# Full setup
import duckdb
import pandas as pd
from sodapy import Socrata
import config
import database_setup as dbs

%reload_ext sql
conn = dbs.get_database_connection(config.DEV_DATABASE)
%sql conn --alias duckdb


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [7]:
client = Socrata("data.ny.gov",
                     config.NY_OPEN_DATA_API_TOKEN,
                     username=config.NY_OPEN_DATA_USERNAME,
                     password=config.NY_OPEN_DATA_PASSWORD)
restriction_type = "timestamp"
restriction = '2023-01-02'
results_v2 = client.get(
    config.MTA_CODE_ORIGIN_DESTINATION_2023,
    where=f"date_trunc_ymd({restriction_type}) = '{restriction}'",
    limit=10000000000
)

ConnectionError: HTTPSConnectionPool(host='data.ny.gov', port=443): Read timed out.

: 

In [14]:
conn.close()

In [15]:
!which python

/Users/Ashwin/opt/anaconda3/envs/capstone/bin/python


In [13]:
with conn:
    conn.sql(f"select * from mta.hourly_ridership_20230102")

In [3]:
timestamp = "'2023-01-02T00:00:00'"
hourly_ridership_20230102_df = dbs.extract_dataset(
    config.MTA_CODE_HOURLY_RIDERSHIP,
    restriction_type="transit_timestamp",
    restriction=timestamp,
    limit=10000000
)

In [5]:
dbs.create_table(conn, hourly_ridership_20230102_df, "mta.hourly_ridership_20230102")

In [10]:
%%sql
select * from information_schema.tables

Running query in 'duckdb'

table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
mta_dev,main,dim_stations,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,main,fct_reduced_fare_2023,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,mta,hourly_ridership_20230102,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,mta,origin_destination_20230102,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,mta,reduced_fare,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,mta,stations,BASE TABLE,None,None,None,None,None,YES,NO,None,None
mta_dev,main,stg_reduced_fare_2023,VIEW,None,None,None,None,None,NO,NO,None,None
mta_dev,main,stg_stations,VIEW,None,None,None,None,None,NO,NO,None,None


In [12]:
%%sql
select * from mta.hourly_ridership_20230102

Running query in 'duckdb'

transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference
2023-01-02T00:00:00.000,subway,43,Parkside Av (Q),Brooklyn,omny,OMNY - Full Fare,7.0,1.0,40.655293,-73.961494,"{'type': 'Point', 'coordinates': [-73.961494, 40.655293]}"
2023-01-02T00:00:00.000,subway,35,"59 St (N,R)",Brooklyn,metrocard,Metrocard - Unlimited 30-Day,4.0,0.0,40.64136,-74.01788,"{'type': 'Point', 'coordinates': [-74.01788, 40.64136]}"
2023-01-02T00:00:00.000,subway,432,"Prospect Av (2,5)",Bronx,omny,OMNY - Full Fare,4.0,0.0,40.819584,-73.90177,"{'type': 'Point', 'coordinates': [-73.90177, 40.819584]}"
2023-01-02T00:00:00.000,subway,357,"Beverly Rd (2,5)",Brooklyn,metrocard,Metrocard - Full Fare,2.0,0.0,40.6451,-73.94896,"{'type': 'Point', 'coordinates': [-73.94896, 40.6451]}"
2023-01-02T00:00:00.000,subway,254,Jamaica-179 St (F),Queens,omny,OMNY - Seniors & Disability,1.0,1.0,40.712646,-73.78381,"{'type': 'Point', 'coordinates': [-73.78381, 40.712646]}"
2023-01-02T00:00:00.000,subway,243,"Church Av (F,G)",Brooklyn,metrocard,Metrocard - Fair Fare,1.0,0.0,40.64404,-73.979675,"{'type': 'Point', 'coordinates': [-73.979675, 40.64404]}"
2023-01-02T00:00:00.000,subway,428,"174 St (2,5)",Bronx,metrocard,Metrocard - Full Fare,4.0,0.0,40.837288,-73.88773,"{'type': 'Point', 'coordinates': [-73.88773, 40.837288]}"
2023-01-02T00:00:00.000,subway,352,New Lots Av (3),Brooklyn,metrocard,Metrocard - Other,3.0,0.0,40.666237,-73.88408,"{'type': 'Point', 'coordinates': [-73.88408, 40.666237]}"
2023-01-02T00:00:00.000,subway,35,"59 St (N,R)",Brooklyn,metrocard,Metrocard - Fair Fare,2.0,0.0,40.64136,-74.01788,"{'type': 'Point', 'coordinates': [-74.01788, 40.64136]}"
2023-01-02T00:00:00.000,subway,243,"Church Av (F,G)",Brooklyn,metrocard,Metrocard - Unlimited 30-Day,1.0,0.0,40.64404,-73.979675,"{'type': 'Point', 'coordinates': [-73.979675, 40.64404]}"
